In [32]:
import sys
sys.path.append("/Users/samy/Desktop/thèse/code/AnomalyDetection/data")
sys.path.append("/Users/samy/Desktop/thèse/code/AnomalyDetection/algorithms/DeepOneClass")

import torch
import json
import os

from mnist import load_datasets, prepare_data_loaders
from model import CNN

In [38]:
ANOMALY_DIGITS = 8
NORMAL_DIGITS = [i for i in range(10)]
NORMAL_DIGITS.remove(ANOMALY_DIGITS)

In [39]:
checkpoint = torch.load(f"models_infos/DeepOneClass/ONE/model_anomaly_{ANOMALY_DIGITS}.pkl")
center = checkpoint["center"]
center = torch.from_numpy(center)

In [40]:
model = CNN()
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [41]:
train_dic_dataset, val_dic_dataset, test_dic_dataset = load_datasets()
normal_val = prepare_data_loaders(train_dic_dataset, val_dic_dataset, NORMAL_DIGITS, batch_size=None)

In [42]:
with torch.no_grad():
    for digit in range(10):
        inputs = test_dic_dataset[digit]
        preds = model(inputs)
        current_batch_size = preds.size(0)
        c_score = center.repeat(current_batch_size, 1)
        dist = torch.sum((preds - c_score)**2, dim=1)
        print(f"Digit : {digit} , mean_score : {torch.mean(dist)*100}")

Digit : 0 , mean_score : 0.1264289915561676
Digit : 1 , mean_score : 0.21490342915058136
Digit : 2 , mean_score : 0.02075709030032158
Digit : 3 , mean_score : 0.010045424103736877
Digit : 4 , mean_score : 0.007852225564420223
Digit : 5 , mean_score : 0.017882637679576874
Digit : 6 , mean_score : 0.0011256918078288436
Digit : 7 , mean_score : 0.00880887359380722
Digit : 8 , mean_score : 0.0026839773636311293
Digit : 9 , mean_score : 0.006968818139284849


In [15]:
inputs_val = normal_val
with torch.no_grad():
    preds_val = model(inputs_val)

In [ ]:
current_batch_size = preds_val.size(0)
c_val = center.repeat(current_batch_size, 1)